In [5]:
# importamos las librerías con las que vamos a trabajar
!pip install psycopg2==2.9.9
!pip install psycopg2-binary==2.9.9
# Trabajar con bases de datos y python
# -----------------------------------------------------------------------
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors


# Trabajar con DataFrames
# -----------------------------------------------------------------------
import pandas as pd
pd.options.display.max_columns = None

In [26]:
df = pd.read_csv("CSVs/Supermercados_limpio.csv", index_col = 0)


In [8]:
#Nos conectamos a postgres
try:
    conexion = psycopg2.connect(
        database = "proyecto4",
        user = "postgres",
        password = "admin",
        host = "localhost",
        port = "5432"

)
except OperationalError:
    print("la contraseña es errónea")

In [10]:
cursor = conexion.cursor()

In [27]:
query_productos = """
    CREATE TABLE IF NOT EXISTS productos (
        id SERIAL PRIMARY KEY,
        nombre VARCHAR(255) NOT NULL,
        categoria VARCHAR(100) NOT NULL
    );
"""

query_supermercados = """
    CREATE TABLE IF NOT EXISTS supermercados (
        id SERIAL PRIMARY KEY,
        nombre VARCHAR(100) NOT NULL UNIQUE
    );
"""

query_precios = """
    CREATE TABLE IF NOT EXISTS precios (
        id SERIAL PRIMARY KEY,
        producto_id INT REFERENCES productos(id),
        supermercado_id INT REFERENCES supermercados(id),
        fecha DATE NOT NULL,
        precio DECIMAL(10, 2) NOT NULL,
        variacion DECIMAL(10, 2),
        variacion_porcentual DECIMAL(5, 2),
        CONSTRAINT unique_price_entry UNIQUE (producto_id, supermercado_id, fecha)
    );
"""

In [ ]:
productos = df[['Nombre_producto', 'Categoría']].drop_duplicates()
for _, row in productos.iterrows():
    cursor.execute("""
        INSERT INTO productos (nombre, categoria)
        VALUES (%s, %s)
        ON CONFLICT (nombre) DO NOTHING;
    """, (row['Nombre_producto'], row['Categoría']))
conn.commit()

supermercados = df['Supermercado'].drop_duplicates()
for supermercado in supermercados:
    cursor.execute("""
        INSERT INTO supermercados (nombre)
        VALUES (%s)
        ON CONFLICT (nombre) DO NOTHING;
    """, (supermercado,))
conn.commit()

# Insertar precios
for _, row in df.iterrows():
    # Obtener los IDs de producto y supermercado
    cursor.execute("SELECT id FROM productos WHERE nombre = %s;", (row['Nombre_producto'],))
    producto_id = cursor.fetchone()[0]

    cursor.execute("SELECT id FROM supermercados WHERE nombre = %s;", (row['Supermercado'],))
    supermercado_id = cursor.fetchone()[0]

    # Insertar el precio
    cursor.execute("""
        INSERT INTO precios (producto_id, supermercado_id, fecha, precio, variacion, variacion_porcentual)
        VALUES (%s, %s, %s, %s, %s, %s)
        ON CONFLICT (producto_id, supermercado_id, fecha) DO NOTHING;
    """, (producto_id, supermercado_id, row['Fecha'], row['Precio'], row['Variación'], row['Variación_porcentual']))
conn.commit()

# Cerrar la conexión
cursor.close()
conn.close()